# Toronto Neighborhood
---
Using both toronto borough information and geospatial coordinates, a k-means algorithm is used to cluster unknown features of the dataset. These features are labelled visually over the neighborhood coordinates.

Install/import libraries used

In [189]:
#!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Web Scraping:

Wikipedia holds a table of all the relevant postal code data. Set url to table address

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Read in HTML from url using beautiful soup. (install the parser used) in this case, we'll use the python parser to read in the table

In [13]:
#!pip install html5lib

data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")

Read in the contents into a meaningful dataframe. Entries without a borough assgined are dropped. Postal codes with multiple neighborhoods are concathenated into one entry. 

In [16]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Verify dimensions of the dataset.

In [53]:
df.shape

(103, 3)

### Geospatial Data:

Using the geocoder api, grab the (lat,lng) coordinates of each neighborhood.

In [ ]:
#!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

If geocoder isn't working, use the geospatial data from the .csv file.

Rename the postal code columns and merge the two dataframes together, matched on the postal code.

In [175]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

toronto_borough = df.merge(geo_data, on=["PostalCode"])
toronto_borough = toronto_borough[toronto_borough['Borough'].str.contains('Toronto', na=False)].reset_index(drop=True)


In [126]:
vtoronto_borough.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


Visualize the toronto neighborhoods with overlaid markers

In [ ]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [76]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [203]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Cluster

Before we cluster, take a look at the density of postal codes within each Borough which will give us a sense of how the clustering algorithm might try and fit

In [198]:
toronto_borough.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude,labels
Borough,,,,,
Central Toronto,9,9,9,9,9
Downtown Toronto,17,17,17,17,17
Downtown Toronto Stn A,1,1,1,1,1
East Toronto,4,4,4,4,4
East Toronto Business,1,1,1,1,1
East York/East Toronto,1,1,1,1,1
West Toronto,6,6,6,6,6


One hot encoding for both "neighborhoods" and "postal codes" for each borough. Even though these variables are mutually exclusive, the density information of each should assist the l-means algorithm to find non-trivial features. 

In [142]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_borough[['Neighborhood','PostalCode']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_borough['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [199]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()

Using K-means clustering for the seven boroughs in question. 

In [200]:
# import k-means from clustering stage
#!pip install sklearn
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

/anaconda3/envs/DataScience/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


array([4, 4, 3, 1, 0, 2, 4], dtype=int32)

In [201]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

Merge column labels with geospatial data

In [ ]:
label_col = toronto_grouped[["Borough","Cluster Labels"]].set_index('Borough')
# using apply function to create a new column
toronto_borough['labels'] = toronto_borough.apply(lambda row: label_col.loc[row["Borough"], "Cluster Labels"], axis = 1)

Overlay the postal codes with label colours

In [202]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighborhood'], toronto_borough['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Conclusion:

inference: It's easy to see that the labels are clustering the density of neighborhoods and postal code density for a set of boroughs